In [1]:
import arcpy
import datetime
import reportlab
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

In [2]:
# Avery 5160 label dimensions and layout (in points; 1 point = 1/72 inch)
label_width = 2.625 * 72  # Label width in points
label_height = 1 * 72  # Label height in points
columns = 3  # Number of columns
rows = 10  # Number of rows
top_margin = 0.5 * 72  # Top margin in points
left_margin = 0.1875 * 72  # Left margin in points
x_gap = 0.125 * 72  # Gap between columns
y_gap = 0  # Gap between rows

In [3]:
working_folder = r'C:\Users\JR067290\OneDrive - Jacobs\Documents\ArcGIS\Projects\Newport_LCRR'
# Set the workspace (change this to your local path)
gdb = r'C:\Users\JR067290\OneDrive - Jacobs\Documents\ArcGIS\Projects\Newport_LCRR\Newport_LCRR.gdb'

arcpy.env.workspace = gdb
# Feature class and related table paths
wServices = f'{gdb}\\wServices'
LCRR_Letter_Tracking = f'{gdb}\\LCRR_Letter_Tracking'

# Get the current date
current_date = datetime.datetime.now().strftime("%Y-%m-%d")  # Formats the date as YYYY-MM-DD

# Output PDF path
output_pdf = working_folder + '\Postcard_' + current_date + '.pdf'

In [4]:
# Initialize PDF canvas
c = canvas.Canvas(output_pdf, pagesize=letter)
width, height = letter
x, y = left_margin, height - top_margin - label_height

# Function to draw an address label
def draw_label(x, y, address):
    c.drawString(x, y, address)

In [5]:
# Determine whether to process selected features or all features
feature_count = arcpy.GetCount_management(wServices)
selection_count = int(feature_count.getOutput(0))
use_selected_features = False

# Check if there are selected features
if selection_count < int(arcpy.GetCount_management(wServices)[0]):
    use_selected_features = True

# Fields to be used from the feature class and related table
feature_fields = ["LetterRelID", "PointAddress"]
table_fields = ["LetterRelID", "LetterSentDate", "LetterType"]

In [6]:
# Start an edit session and operation
with arcpy.da.Editor(arcpy.env.workspace) as editor:
    # Create a SearchCursor for the feature class
    with arcpy.da.SearchCursor(wServices, feature_fields, None if use_selected_features else "LetterRelID IS NOT NULL") as cursor:
        # Open an InsertCursor to add records to the related table
        with arcpy.da.InsertCursor(LCRR_Letter_Tracking, table_fields) as insert_cursor:
            for row in cursor:
                # Create a new row for the related table
                new_row = (row[0], current_date, "Postcard")  # Include LetterRelID, current date, and set LetterType to "Postcard"
                # Insert the new row into the related table
                insert_cursor.insertRow(new_row)

                 # Draw address label on PDF
                address = row[1]
                draw_label(x, y, address)
                x += label_width + x_gap
                if x + label_width > width - left_margin:
                    x = left_margin
                    y -= label_height + y_gap
                    if y < top_margin:
                        c.showPage()
                        y = height - top_margin - label_height

# Save the PDF
c.save()


print("Operation completed: Records added and PDF generated.")

Operation completed: Records added and PDF generated.


In [ ]:
PointAddress

